## 구글드라이브 마운트

In [36]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


## 필요 라이브러리 임포트
- 추가 사용자 정의함수 불러오기

In [37]:
# 라이브러리 임포트
import os
import numpy as np
import pandas as pd
import chardet

import time

import folium

# 사용자 정의함수 경로 설정
import sys
# sys.path.append("H:/내 드라이브/_Projects/재정데이터분석_미니프로젝트/Source/_Custom_Function")
sys.path.append("/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/Source/_Custom_Function")

# 사용자 정의함수 불러오기
import ExcelFile_Merge as em
import ExcelFile_Read as er
import ExcelFile_EDA as ee

import dataset_function as ds_f
import folium_function as fl_f
import plot_function as pl_f

In [38]:
# 나의 루트 경로 설정
path_dateset = '/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/DataSet/'

In [39]:
file_name_1 = '전처리/우리 지자체 예산서(완성)v1.0.xlsx'

savefile_name_1 = '우리 지자체 예산서(완성)v1.0(전처리)).csv'

## 전처리 할 파일 선택

In [40]:
file_name = file_name_1
savefile_name = savefile_name_1

In [41]:
df_temp = em.Read_File(path_dateset + file_name)

In [42]:
df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   No           243 non-null    int64  
 1   지역명          243 non-null    object 
 2   자치단체코드       243 non-null    int64  
 3   자치단체명        243 non-null    object 
 4   도로           243 non-null    float64
 5   대중교통･ 물류등기타  243 non-null    float64
 6   합계           243 non-null    float64
 7   URL          243 non-null    object 
dtypes: float64(3), int64(2), object(3)
memory usage: 15.3+ KB


In [43]:
df_temp

,No,지역명,자치단체코드,자치단체명,도로,대중교통･ 물류등기타,합계,URL
0,1,서울,1100000,서울본청,1.94,6.93,8.87,https://news.seoul.go.kr/gov/archives/554900?l...
1,2,서울,1111000,서울종로구,1.79,6.19,7.98,https://www.jongno.go.kr/portal/bbs/selectBoar...
2,3,서울,1112000,서울중구,0.86,1.75,2.61,https://www.junggu.seoul.kr/content.do?cmsid=1...
3,4,서울,1113000,서울용산구,2.33,0.47,2.80,https://www.yongsan.go.kr/portal/bbs/B0000035/...
4,5,서울,1114000,서울성동구,0.78,2.15,2.93,https://www.sd.go.kr/main/selectBbsNttList.do?...
...,...,...,...,...,...,...,...,...
238,239,경남,4877000,경남산청군,2.42,1.21,3.63,https://www.sancheong.go.kr/www/contents.do?ke...
239,240,경남,4878000,경남함양군,1.85,1.74,3.59,https://www.hygn.go.kr/00430/00586/05832.web
240,241,경남,4879000,경남거창군,2.15,2.32,4.47,https://budget.geochang.go.kr
241,242,경남,4880000,경남합천군,2.93,1.57,4.50,https://www.hc.go.kr/04959/04965/09475.web


In [44]:
df_temp['시도'] = df_temp['자치단체명'].str.slice(start=0, stop = 2)
df_temp['시군구'] = df_temp['자치단체명'].str.slice(start=2)

In [45]:
df_temp.head(3)

,No,지역명,자치단체코드,자치단체명,도로,대중교통･ 물류등기타,합계,URL,시도,시군구
0,1,서울,1100000,서울본청,1.94,6.93,8.87,https://news.seoul.go.kr/gov/archives/554900?l...,서울,본청
1,2,서울,1111000,서울종로구,1.79,6.19,7.98,https://www.jongno.go.kr/portal/bbs/selectBoar...,서울,종로구
2,3,서울,1112000,서울중구,0.86,1.75,2.61,https://www.junggu.seoul.kr/content.do?cmsid=1...,서울,중구


In [46]:
_columns = ['시도', '시군구', '도로', '대중교통･ 물류등기타', '합계']
df_temp_2 = df_temp[_columns]

In [47]:
df_temp_2.shape

(243, 5)

In [48]:
df_temp_2.head(3)

,시도,시군구,도로,대중교통･ 물류등기타,합계
0,서울,본청,1.94,6.93,8.87
1,서울,종로구,1.79,6.19,7.98
2,서울,중구,0.86,1.75,2.61


In [49]:
cond1 = df_temp_2['시군구'].str.contains('본청')
cond2 = df_temp_2['시도'].isin(['세종', '제주'])
df_temp_3 = df_temp_2[~(cond1 & ~cond2)]
df_temp_3

,시도,시군구,도로,대중교통･ 물류등기타,합계
1,서울,종로구,1.79,6.19,7.98
2,서울,중구,0.86,1.75,2.61
3,서울,용산구,2.33,0.47,2.80
4,서울,성동구,0.78,2.15,2.93
5,서울,광진구,1.76,6.32,8.08
...,...,...,...,...,...
238,경남,산청군,2.42,1.21,3.63
239,경남,함양군,1.85,1.74,3.59
240,경남,거창군,2.15,2.32,4.47
241,경남,합천군,2.93,1.57,4.50


In [50]:
_val_before = ['경기', '서울', '전남', '경북', '강원', '경남', '부산', '충남', '전북', '충북', '인천', '대구', '광주', '대전', '울산', '세종', '제주']
_val_after = ['경기도', '서울특별시', '전라남도', '경상북도', '강원특별자치도', '경상남도', '부산광역시', '충청남도', '전북특별자치도', '충청북도', '인천광역시', '대구광역시', '대전광역시', '광주광역시', '울산광역시', '세종특별자치시', '제주특별자치도']

for i in range(len(_val_before)):
  # print(_val_before[i])
  df_temp_3['시도'][df_temp_3['시도'] == _val_before[i]] = _val_after[i]

<ipython-input-50-74c1c0829066>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_temp_3['시도'][df_temp_3['시도'] == _val_before[i]] = _val_after[i]
<ipython-input-50-74c1c0829066>:6: SettingWithCopyWarning: 
A value is trying to be set on a 

In [51]:
df_temp_3.head(5)

,시도,시군구,도로,대중교통･ 물류등기타,합계
1,서울특별시,종로구,1.79,6.19,7.98
2,서울특별시,중구,0.86,1.75,2.61
3,서울특별시,용산구,2.33,0.47,2.80
4,서울특별시,성동구,0.78,2.15,2.93
5,서울특별시,광진구,1.76,6.32,8.08


In [56]:
df_temp_3['시군구'][df_temp_3['시도'].isin(['세종특별자치시'])] = '세종특별자치시'
df_temp_3['시군구'][df_temp_3['시도'].isin(['제주특별자치도'])] = '제주특별자치도'

<ipython-input-56-10751ebeb9b1>:1: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_temp_3['시군구'][df_temp_3['시도'].isin(['세종특별자치시'])] = '세종특별자치시'
<ipython-input-56-10751ebeb9b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [57]:
df_temp_3[df_temp_3['시도'].isin(['세종특별자치시', '제주특별자치도'])]

,시도,시군구,도로,대중교통･ 물류등기타,합계
82,세종특별자치시,세종특별자치시,1.20,5.32,6.52
242,제주특별자치도,제주특별자치도,2.66,3.79,6.45


In [58]:
df_temp_3.rename(columns = {'대중교통･ 물류등기타' : '대중교통_물류등기타'}, inplace = True)

<ipython-input-58-44a5da71031e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp_3.rename(columns = {'대중교통･ 물류등기타' : '대중교통_물류등기타'}, inplace = True)


In [59]:
df_temp_3.head(5)

,시도,시군구,도로,대중교통_물류등기타,합계
1,서울특별시,종로구,1.79,6.19,7.98
2,서울특별시,중구,0.86,1.75,2.61
3,서울특별시,용산구,2.33,0.47,2.80
4,서울특별시,성동구,0.78,2.15,2.93
5,서울특별시,광진구,1.76,6.32,8.08


In [60]:
df_temp_3.shape

(228, 5)

In [61]:
path_dateset + savefile_name

'/gdrive/MyDrive/_Projects/재정데이터분석_미니프로젝트/DataSet/우리 지자체 예산서(완성)v1.0(전처리)).csv'

In [62]:
df_temp_3.to_csv(path_dateset + savefile_name, index = False, encoding='utf-8-sig')

In [63]:
pd.read_csv(path_dateset + savefile_name)

,시도,시군구,도로,대중교통_물류등기타,합계
0,서울특별시,종로구,1.79,6.19,7.98
1,서울특별시,중구,0.86,1.75,2.61
2,서울특별시,용산구,2.33,0.47,2.80
3,서울특별시,성동구,0.78,2.15,2.93
4,서울특별시,광진구,1.76,6.32,8.08
...,...,...,...,...,...
223,경상남도,산청군,2.42,1.21,3.63
224,경상남도,함양군,1.85,1.74,3.59
225,경상남도,거창군,2.15,2.32,4.47
226,경상남도,합천군,2.93,1.57,4.50
